In [ ]:
import hashlib
import json
import time
from cryptography.fernet import Fernet
import psutil  

class Blockchain:
    def __init__(self):
        self.chain = []
        self.current_transactions = []
        self.authorized_devices = set()
        self.new_block(previous_hash='1', proof=100)  
    def new_block(self, proof, previous_hash=None):
        block = {
            'index': len(self.chain) + 1,
            'timestamp': time.time(),
            'transactions': self.current_transactions,
            'proof': proof,
            'previous_hash': previous_hash or self.hash(self.chain[-1]),
        }
        self.current_transactions = []
        self.chain.append(block)
        return block

    def new_transaction(self, sender, recipient, amount):
        self.current_transactions.append({
            'sender': sender,
            'recipient': recipient,
            'amount': amount,
        })
        return self.last_block['index'] + 1

    def authorize_device(self, mac_address):
        self.authorized_devices.add(mac_address)
        return True

    def is_device_authorized(self, mac_address):
        return mac_address in self.authorized_devices

    def record_activity(self, mac_address, status):
        self.new_transaction(sender="network_monitor", recipient=mac_address, amount=1)
        print(f'Recorded activity for {mac_address}: {status}')

    def record_data_packet(self, sender_mac, recipient_mac, data):
        data_hash = hashlib.sha256(data.encode()).hexdigest()
        self.new_transaction(sender=sender_mac, recipient=recipient_mac, amount=1)
        print(f"Data packet from {sender_mac} to {recipient_mac} recorded with hash: {data_hash}")

    def generate_session_key(self):
        key = Fernet.generate_key()
        print(f"Generated Session Key: {key.decode()}")
        return key

    def encrypt_data(self, data, session_key):
        fernet = Fernet(session_key)
        encrypted_data = fernet.encrypt(data.encode())
        return encrypted_data

    def decrypt_data(self, encrypted_data, session_key):
        fernet = Fernet(session_key)
        decrypted_data = fernet.decrypt(encrypted_data).decode()
        return decrypted_data

    @staticmethod
    def hash(block):
        block_string = json.dumps(block, sort_keys=True).encode()
        return hashlib.sha256(block_string).hexdigest()

    @property
    def last_block(self):
        return self.chain[-1]

    def proof_of_work(self, last_proof):
        proof = 0
        while self.valid_proof(last_proof, proof) is False:
            proof += 1
        return proof

    @staticmethod
    def valid_proof(last_proof, proof):
        guess = f'{last_proof}{proof}'.encode()
        guess_hash = hashlib.sha256(guess).hexdigest()
        return guess_hash[:4] == "0000"

   
    def measure_resources(self):
        process = psutil.Process()
        cpu_usage = process.cpu_percent(interval=1)
        memory_usage = process.memory_info().rss / (1024 * 1024) 
        return cpu_usage, memory_usage

     
    def check_block_integrity(self):
        for i in range(1, len(self.chain)):
            if self.chain[i]['previous_hash'] != self.hash(self.chain[i - 1]):
                return False
        return True

    def transaction_success_rate(self):
        successful_transactions = len(self.chain[-1]['transactions'])  
        total_transactions = len(self.current_transactions)  
        return (successful_transactions / total_transactions) * 100 if total_transactions > 0 else 0


blockchain = Blockchain()

blockchain_start_time = time.time()
proof = blockchain.proof_of_work(blockchain.last_block['proof'])
blockchain.new_block(proof)
blockchain_end_time = time.time()

blockchain_time = blockchain_end_time - blockchain_start_time
print(f"Blockchain block creation time: {blockchain_time:.6f} seconds")


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import time

file_path = 'dataset.csv'
data = pd.read_csv(file_path)

features = data.drop(columns=['label'])

scaler = StandardScaler()
X_scaled = scaler.fit_transform(features)

X_train, X_test = train_test_split(X_scaled, test_size=0.3, random_state=42)

svm_model = OneClassSVM(kernel='rbf', gamma=0.005, nu=0.01)

isolation_model = IsolationForest(n_estimators=100, contamination=0.02, random_state=42)

start_train_time = time.time()
svm_model.fit(X_train)
isolation_model.fit(X_train)
end_train_time = time.time()
training_time = end_train_time - start_train_time
print(f"Training Time: {training_time:.6f} seconds")

num_anomalies = 100  
anomalies = []

for _ in range(num_anomalies):
    random_sample = X_test[np.random.randint(0, X_test.shape[0])]
    large_noise = np.random.normal(0, 7, size=random_sample.shape)
    anomaly_sample = random_sample + large_noise
    anomalies.append(anomaly_sample)

X_test_with_anomalies = np.concatenate([X_test, anomalies])

y_true = np.concatenate([np.zeros(len(X_test)), np.ones(num_anomalies)])

start_pred_time = time.time()

svm_predictions = svm_model.predict(X_test_with_anomalies)
isolation_predictions = isolation_model.predict(X_test_with_anomalies)

end_pred_time = time.time()
prediction_time = end_pred_time - start_pred_time
print(f"Prediction Time: {prediction_time:.6f} seconds")

svm_predictions = np.where(svm_predictions == -1, 1, 0)
isolation_predictions = np.where(isolation_predictions == -1, 1, 0)

final_predictions = (svm_predictions + isolation_predictions) // 2

accuracy = accuracy_score(y_true, final_predictions)
print(f"Accuracy of the Hybrid Model (One-Class SVM & Isolation Forest): {accuracy:.2f}")

print("\nClassification Report:")
print(classification_report(y_true, final_predictions, zero_division=0))

conf_matrix = confusion_matrix(y_true, final_predictions)

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", cbar=True)
plt.title("Confusion Matrix")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

print("\nConfusion Matrix:")
print(conf_matrix)

tn, fp, fn, tp = confusion_matrix(y_true, final_predictions).ravel()
false_positive_rate = fp / (fp + tn)
false_negative_rate = fn / (fn + tp)
print(f"False Positive Rate: {false_positive_rate:.2f}")
print(f"False Negative Rate: {false_negative_rate:.2f}")


In [ ]:
def check_device_identity(blockchain, mac_address):
    start_time = time.time()
    is_authorized = blockchain.is_device_authorized(mac_address)
    end_time = time.time()

    verification_time = end_time - start_time  
    print(f"Device verification time for {mac_address}: {verification_time:.6f} seconds")

    return is_authorized, verification_time

new_devices = [
    {"mac_address": "00:1A:2B:3C:4D:5E", "is_authorized": True},
    {"mac_address": "11:22:33:44:55:66", "is_authorized": False},
    {"mac_address": "AA:BB:CC:DD:EE:FF", "is_authorized": True},
]

for device in new_devices:
    if device['is_authorized']:
        blockchain.authorize_device(device['mac_address'])

print("\nTesting Devices:")
for idx, device in enumerate(new_devices):
    mac_address = device['mac_address']
    
    is_authorized, verification_time = check_device_identity(blockchain, mac_address)

    is_normal = final_predictions[idx] == 0  

    if is_authorized and is_normal:
        status = 'authorized and behaves normally'
        print(f'Device {mac_address} is {status}.')

        channel_creation_start = time.time()
        session_key = blockchain.generate_session_key()
        encrypted_data = blockchain.encrypt_data("This is a secure data packet.", session_key)
        channel_creation_end = time.time()

        channel_creation_time = channel_creation_end - channel_creation_start
        print(f"Channel creation time: {channel_creation_time:.6f} seconds")

        blockchain.record_data_packet(sender_mac=mac_address, recipient_mac="network_monitor", data="secure_packet")

    elif not is_authorized:
        status = 'NOT authorized'
        print(f'Device {mac_address} is {status}.')
    else:
        status = 'authorized but shows abnormal behavior'
        print(f'Device {mac_address} is {status}.')

    blockchain.record_activity(mac_address, status)


In [ ]:
cpu_usage, memory_usage = blockchain.measure_resources()
print(f"CPU Usage: {cpu_usage}%")
print(f"Memory Usage: {memory_usage:.2f} MB")

start_encryption_time = time.time()

session_key = blockchain.generate_session_key()
encrypted_data = blockchain.encrypt_data("This is a secure data packet.", session_key)

end_encryption_time = time.time()
encryption_time = end_encryption_time - start_encryption_time

start_decryption_time = time.time()
decrypted_data = blockchain.decrypt_data(encrypted_data, session_key)
end_decryption_time = time.time()
decryption_time = end_decryption_time - start_decryption_time

print(f"Encryption time for secure packet: {encryption_time:.6f} seconds")
print(f"Decryption time for secure packet: {decryption_time:.6f} seconds")

device_verification_start = time.time()

device_mac = "00:1A:2B:3C:4D:5E"
is_device_authorized = blockchain.is_device_authorized(device_mac)

device_verification_end = time.time()
device_verification_time = device_verification_end - device_verification_start

print(f"Device verification time: {device_verification_time:.6f} seconds")

channel_creation_start = time.time()

session_key = blockchain.generate_session_key()
encrypted_data = blockchain.encrypt_data("Secure communication data", session_key)

channel_creation_end = time.time()
channel_creation_time = channel_creation_end - channel_creation_start

print(f"Channel creation time: {channel_creation_time:.6f} seconds")

blockchain_start_time = time.time()
proof = blockchain.proof_of_work(blockchain.last_block['proof'])
blockchain.new_block(proof)
blockchain_end_time = time.time()

blockchain_time = blockchain_end_time - blockchain_start_time
print(f"Blockchain block creation time: {blockchain_time:.6f} seconds")

total_processing_time = encryption_time + decryption_time + device_verification_time + channel_creation_time + blockchain_time
print(f"Total processing time: {total_processing_time:.6f} seconds")

print("\n--- Final Results for Phase 4 ---")
print(f"Encryption time for secure packet: {encryption_time:.6f} seconds")
print(f"Decryption time for secure packet: {decryption_time:.6f} seconds")
print(f"Device verification time: {device_verification_time:.6f} seconds")
print(f"Channel creation time: {channel_creation_time:.6f} seconds")
print(f"Blockchain block creation time: {blockchain_time:.6f} seconds")
print(f"Total processing time: {total_processing_time:.6f} seconds")
print(f"CPU Usage: {cpu_usage}%")
print(f"Memory Usage: {memory_usage:.2f} MB")

speed = blockchain_time  
print(f"Speed (block creation time): {speed:.6f} seconds")

is_safe = blockchain.check_block_integrity()  
print(f"Safety (blockchain integrity): {'Safe' if is_safe else 'Not Safe'}")

cpu_usage, memory_usage = blockchain.measure_resources()  
print(f"Efficiency (CPU Usage): {cpu_usage}%")
print(f"Efficiency (Memory Usage): {memory_usage:.2f} MB")

time_spent = blockchain_time  
print(f"Time Spent: {time_spent:.6f} seconds")
